In [129]:
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import func, and_

from database.database import Database
from database.models import Brand, Receipt, Transaction, User

db = Database()

def start_of_month(date, month_shift=0):
    """Return the first day of the month for the given date."""
    year = date.year + (date.month + month_shift - 1) // 12
    month = (date.month + month_shift - 1) % 12 + 1
    return datetime(year, month, 1)

def get_recent_user_creation(db):
    with db.Session() as session:
        query = (
            session.query(
                User.created_date,
            )
            .select_from(User)
            .order_by(User.created_date.desc())
            .limit(1)
        )
        results = query.all()
        return results[0][0]

# There are no recent users
fake_date_now = get_recent_user_creation(db)
print(f"Pretending today is {fake_date_now}")

INFO:database.database:Connection successful


Database initialized
Pretending today is 2021-02-12 08:11:06.240000


# Top 5 Brands by Receipts Scanned

What are the top 5 brands by receipts scanned for most recent month?

In [138]:
def brand_with_most(db, sort_by_col, col_label=None, start_date=None, end_date=None, limit=5):
    """Determine top brand based on by_col
    by_col: sql column, for example func.sum(Receipt.total_spent) for total spent
    num_months is used to only consider users in the last num_months"""
    with db.Session() as session:

        query = (
            session.query(
                Brand.name,
                Brand.brand_code,
                sort_by_col.label(col_label)
            )
            .select_from(Transaction)
            .join(Receipt, Receipt.id == Transaction.receipt_id)
            .join(Brand, Brand.brand_code == Transaction.brand_code)
            .join(User, User.id == Receipt.user_id)
            .group_by(Brand.name, Brand.brand_code)
            .order_by(sort_by_col.desc())
        )
        
        if start_date:
            query = query.filter(User.created_date >= start_date)
        if end_date:
            query = query.filter(User.created_date <= end_date)
            
        query = query.limit(limit)
        
        results = pd.read_sql_query(query.statement, session.bind)
        return results
    

top_5_brands_last = brand_with_most(db,
                                    sort_by_col=func.count(Receipt.id), 
                                    col_label="total_receipts_scanned",
                                    start_date=start_of_month(fake_date_now, -1), 
                                    end_date=start_of_month(fake_date_now, 0), 
                                    limit=5)
top_5_brands_last

,name,brand_code,total_receipts_scanned
0,Pepsi,PEPSI,74
1,Kleenex,KLEENEX,70
2,KNORR,KNORR,60
3,Doritos,DORITOS,55
4,Kraft,KRAFT,28


# Change Top 5 Brands by Receipts Scanned

How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for
 the previous month?

In [141]:
top_5_brands_last = brand_with_most(db,
                                    sort_by_col=func.count(Receipt.id), 
                                    col_label="total_receipts_scanned",
                                    start_date=start_of_month(fake_date_now, -2), 
                                    end_date=start_of_month(fake_date_now, -1), 
                                    limit=5)
top_5_brands_last

,name,brand_code,total_receipts_scanned


# Average spend from receipts
When considering *average spend* from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’,
 which is greater?

In [132]:
def compare_average_spend(db):
    with db.Session() as session:
        query = (
            session.query(
                Receipt.rewards_receipt_status,
                func.avg(Receipt.total_spent).label('average_spend')
            )
            .select_from(Receipt)
            .filter(Receipt.rewards_receipt_status.in_(['FINISHED', 'REJECTED']))
            .group_by(Receipt.rewards_receipt_status)
        )

        results = pd.read_sql_query(query.statement, session.bind)
    return results

df_avg_spent = compare_average_spend(db)
display(df_avg_spent)

,rewards_receipt_status,average_spend
0,REJECTED,23.326056
1,FINISHED,80.854305


Assuming FINISHED means accepted: accepted receipts have about $60 more then rejected receipts 

# Total Items Purchased

When considering *total number of items purchased* from receipts with 'rewardsReceiptStatus’ of
‘Accepted’ or ‘Rejected’, which is greater?

In [133]:
def compare_total_items(db):
    with db.Session() as session:
        query = (
            session.query(
                Receipt.rewards_receipt_status,
                func.sum(Receipt.purchased_item_count).label('total_count')
            )
            .select_from(Receipt)
            .filter(Receipt.rewards_receipt_status.in_(['FINISHED', 'REJECTED']))
            .group_by(Receipt.rewards_receipt_status)
        )

        results = pd.read_sql_query(query.statement, session.bind)
    return results

df_total = compare_total_items(db)
display(df_total)

,rewards_receipt_status,total_count
0,REJECTED,173
1,FINISHED,8184


Significantly more items purcharse from accepted receipts.

# Brand with most spend
Which brand has the most *spend* among users who were created within the past 6 months?

In [142]:
brands = brand_with_most(db,
                         sort_by_col=func.sum(Receipt.total_spent), 
                         col_label="total_spent",
                         start_date=start_of_month(fake_date_now, -6), 
                         end_date=start_of_month(fake_date_now, 0), 
                         limit=5)
brands

,name,brand_code,total_spent
0,Pepsi,PEPSI,78870.86
1,Kraft,KRAFT,70858.88
2,KNORR,KNORR,61157.05
3,Kleenex,KLEENEX,56050.44
4,Doritos,DORITOS,48321.46


# Brand with most transactions
Which brand has the most *transactions* among users who were created within the past 6 months?

In [143]:
brands = brand_with_most(db,
                         sort_by_col=func.count(Transaction.id), 
                         col_label="total_transactions",
                         start_date=start_of_month(fake_date_now, -6), 
                         end_date=start_of_month(fake_date_now, 0), 
                         limit=5)
brands

,name,brand_code,total_transactions
0,Pepsi,PEPSI,74
1,Kleenex,KLEENEX,70
2,KNORR,KNORR,60
3,Doritos,DORITOS,55
4,Kraft,KRAFT,28
